# Introduction

In this notebook we demonstrate the use of **LSI (Latent Semantic Indexing)** technique of Information Retrieval context to make trace link recovery between Test Cases and Bug Reports.

We model our study as follows:

* Each bug report title, summary and description compose a single query.
* We use each use case content as an entire document that must be returned to the query made

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.externals.joblib import Parallel, delayed
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from utils import plots
from utils import firefox_dataset as fd
from utils import tokenizers as tok
from utils import aux_functions
from utils import model_evaluator as m_eval

from models.lsi import LSI
from models.model_hyperps import LSI_Model_Hyperp

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import warnings; warnings.simplefilter('ignore')

## Load Dataset

In [3]:
test_cases_df = fd.read_testcases_df()
bug_reports_df = fd.read_bugreports_df()

corpus = test_cases_df.tc_desc
query = bug_reports_df.br_desc

test_cases_names = test_cases_df.tc_name
bug_reports_names = bug_reports_df.br_name

orc = fd.read_trace_df()
orc.set_index('tc_name', inplace=True, drop=True)

print(bug_reports_df.shape)
print(test_cases_df.shape)
print(orc.shape)

(35314, 12)
(207, 10)
(207, 35314)


## Evaluate Recovering Efficiency

In order to evaluate the efficiency of the algorithm tested (LSI), we use common metrics applied in the field of IR:

    * Precision
    * Recall
    * F1-score

### Analysis with Default Values of LSI Model

In [12]:
%%time

best_model = LSI()
best_model.recover_links(corpus, query, test_cases_names, bug_reports_names)

NameError: name 'pd' is not defined

In [13]:
best_model.get_trace_links_df().shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
%%time

evaluator = m_eval.ModelEvaluator(orc, best_model)
evaluator.evaluate_model(verbose=True)
#evaluator.plot_precision_vs_recall()

### Try Specific Model - Estimated Best Hyperparameters

##### Get Dataset Subsets

In [3]:
bugreports_subset_df = bug_reports_df[(bug_reports_df.Version == '48 Branch') | (bug_reports_df.Version == '60 Branch')].sample(15, random_state=42)
testcases_subset_df = test_cases_df[(test_cases_df.TestDay.str.contains('20161014')) | (test_cases_df.TestDay.str.contains('20161028'))].sample(10, random_state=1000)

selected_testcases = ['TC_{}_TRG'.format(tc_num) for tc_num in [13, 14, 15, 16, 17, 18]]  # should link with 48 Branch
aux_tc = test_cases_df[test_cases_df.tc_name.isin(selected_testcases)]

tc_subset_df = testcases_subset_df.append(aux_tc)
tc_subset_df.drop_duplicates(inplace=True)

corpus_subset = tc_subset_df.tc_desc
query_subset = bugreports_subset_df.br_desc
testcases_names_subset = tc_subset_df.tc_name
bug_reports_names_subset = bugreports_subset_df.br_name
orc_subset_df = orc.loc[testcases_names_subset, bug_reports_names_subset]

print('TestCases Subset Shape: {}'.format(tc_subset_df.shape))
print('BugReports Subset Shape: {}'.format(bugreports_subset_df.shape))
print('Oracle Subset Shape: {}'.format(orc_subset_df.shape))

TestCases Subset Shape: (14, 10)
BugReports Subset Shape: (15, 12)
Oracle Subset Shape: (14, 15)


#### Checking Content of Bugs

In [15]:
bugreports_subset_df[bugreports_subset_df.Version == '48 Branch'].head()

,Bug_Number,Summary,Platform,Component,Version,Creation_Time,Whiteboard,QA_Whiteboard,First_Comment_Text,First_Comment_Creation_Time,br_name,br_desc
11303,1268934,Reducing tab height causes a gap above tabs,Unspecified,Theme,48 Branch,2016-04-29T17:43:31Z,NaN,NaN,Created attachment 8747147 output.webm User A...,2016-04-29T17:43:31Z,BR_1268934_SRC,1268934 Reducing tab height causes a gap above...
12186,1282551,"CTRL+G finds text hidden in a drop down menu, ...",Unspecified,Search,48 Branch,2016-06-27T19:30:07Z,NaN,NaN,User Agent: Mozilla/5.0 (X11; Linux i686; rv:4...,2016-06-27T19:30:07Z,BR_1282551_SRC,1282551 CTRL+G finds text hidden in a drop dow...
12729,1291175,"revert new ""awesome"" bar in FF48",Unspecified,Untriaged,48 Branch,2016-08-02T05:52:12Z,NaN,NaN,User Agent: Mozilla/5.0 (Windows NT 6.1; Win64...,2016-08-02T05:52:12Z,BR_1291175_SRC,"1291175 revert new ""awesome"" bar in FF48 Unspe..."
13465,1299787,Impossible to set browser.urlbar.maxRichResult...,Unspecified,Untriaged,48 Branch,2016-09-01T12:49:52Z,NaN,NaN,User Agent: Mozilla/5.0 (X11; Ubuntu; Linux i6...,2016-09-01T12:49:52Z,BR_1299787_SRC,1299787 Impossible to set browser.urlbar.maxRi...


In [16]:
testcases_subset_df[testcases_subset_df.TestDay.str.contains('20160603')].head()

,TC_Number,TestDay,Gen_Title,Crt_Nr,Title,Preconditions,Steps,Expected_Result,tc_name,tc_desc
18,19,20160603 + 20160624 + 20161014,new awesomebar tests - awesome bar icons left,7,URL Bar - Globe Icon,NaN,1. Verify the Globe icon displays for Firefox ...,1. The globe is correctly displayed.,TC_19_TRG,19 20160603 + 20160624 + 20161014 new awesomeb...
15,16,20160603 + 20160624 + 20161014,new awesomebar tests - awesome bar search,4,Search State - Favorites,NaN,1. Ensure the star appears if the drop down di...,1. Search result has the star icon displayed\n...,TC_16_TRG,16 20160603 + 20160624 + 20161014 new awesomeb...
14,15,20160603 + 20160624 + 20161014,new awesomebar tests - awesome bar search,3,Search State - Favicons/Search Indicator,NaN,1. Ensure Favicons for sites are displayed - E...,1. Favicon or search indicator icon is display...,TC_15_TRG,15 20160603 + 20160624 + 20161014 new awesomeb...


In [4]:
all_hyperparams = {
    LSI_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : [('cosine' ,x)  for x in [.80]],
    LSI_Model_Hyperp.TOP.value : [10],
    LSI_Model_Hyperp.SVD_MODEL_N_COMPONENTS.value: [100],
    LSI_Model_Hyperp.VECTORIZER_NGRAM_RANGE.value: [(1,1)],
    LSI_Model_Hyperp.VECTORIZER.value : [TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)],
    LSI_Model_Hyperp.VECTORIZER_TOKENIZER.value : [tok.WordNetBased_LemmaTokenizer()]
}

hyperparams = aux_functions.generate_params_comb_list(**all_hyperparams)          

print('Performing model hyperparameters search...')

def run_model(idx, **hyperp):    
    current_model = LSI(**hyperp)
    current_model.set_name('LSI_Model_{}'.format(idx))
    current_model.recover_links(corpus_subset, query_subset, testcases_names_subset, bug_reports_names_subset)
    
    evaluator = m_eval.ModelEvaluator(orc_subset_df, current_model)
    evaluator.evaluate_model()
    evaluator.dump_model()
    evaluator.dump_evaluator()
    
    return([evaluator.get_mean_precision(), 
            evaluator.get_mean_recall(),
            evaluator.get_mean_fscore(), 
            evaluator.get_model().get_name(),
            evaluator.get_model().get_top_value(),
            evaluator.get_model().get_vectorizer_type(), 
            evaluator.get_model().get_tokenizer_type(),
            evaluator.get_model().get_sim_measure_min_threshold()[0],
            evaluator.get_model().get_sim_measure_min_threshold()[1],
            evaluator.get_model().get_model_dump_path(),
            evaluator.get_evaluator_dump_path()
           ])

tasks = [(idx,hp) for idx,hp in enumerate(hyperparams)]
#results = Parallel(n_jobs=-1, verbose=1)(delayed(run_model)(idx, **hp) for idx,hp in tasks)
results = [run_model(t[0],**t[1]) for t in tasks]
results_df = pd.DataFrame(data=results, 
                          columns=['precision', 'recall', 'fscore', 'model_name', 'top_value', 'vectorizer', 'tokenizer', 'metric', 'metric_value', 'model_dump', 'evaluator_dump'])
results_df = results_df.astype(dtype={'model_dump' : str, 'evaluator_dump' : str})

bm = aux_functions.report_best_model(results_df)
bm.save_sim_matrix()

aux_functions.highlight_df(bm.get_trace_links_df())

Performing model hyperparameters search...
------------ Report -------------------

Total of Analyzed Hyperparameters Combinations: 1

Best Model Hyperparameters Combination Found:

{'Measures': {'Mean FScore of LSI_Model_0': 0.016666666666666666,
              'Mean Precision of LSI_Model_0': 0.06666666666666667,
              'Mean Recall of LSI_Model_0': 0.009523809523809523},
 'Setup': [{'Name': 'LSI_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.8)},
           {'Top Value': 10},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 100,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.float64'>,
                           'encoding': 'utf-8'

br_name,BR_1268934_SRC,BR_1463735_SRC,BR_1432520_SRC,BR_1497738_SRC,BR_1463274_SRC,BR_1461828_SRC,BR_1443754_SRC,BR_1443632_SRC,BR_1282551_SRC,BR_1418983_SRC,BR_1291175_SRC,BR_1513270_SRC,BR_1450216_SRC,BR_1436749_SRC,BR_1299787_SRC
tc_name,,,,,,,,,,,,,,,
TC_165_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_19_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_152_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_16_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_160_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_169_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_145_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_15_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_149_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
aux_functions.highlight_df(orc_subset_df)

,BR_1268934_SRC,BR_1463735_SRC,BR_1432520_SRC,BR_1497738_SRC,BR_1463274_SRC,BR_1461828_SRC,BR_1443754_SRC,BR_1443632_SRC,BR_1282551_SRC,BR_1418983_SRC,BR_1291175_SRC,BR_1513270_SRC,BR_1450216_SRC,BR_1436749_SRC,BR_1299787_SRC
tc_name,,,,,,,,,,,,,,,
TC_165_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_19_TRG,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1
TC_152_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_16_TRG,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1
TC_160_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_169_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_145_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_15_TRG,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1
TC_149_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Find The Best Model

In [22]:
all_hyperparams = {
    LSI_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : [('cosine' ,x)  for x in [.75,.85,.95]],
    LSI_Model_Hyperp.TOP.value : [10],
    LSI_Model_Hyperp.SVD_MODEL_N_COMPONENTS.value: [5,10],
    LSI_Model_Hyperp.VECTORIZER_NGRAM_RANGE.value: [(1,1), (1,2)],
    LSI_Model_Hyperp.VECTORIZER.value : [TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True), 
                         CountVectorizer(stop_words='english')],
    LSI_Model_Hyperp.VECTORIZER_TOKENIZER.value : [tok.PorterStemmerBased_Tokenizer(), tok.LancasterStemmerBased_Tokenizer(), 
                                                   tok.WordNetBased_LemmaTokenizer(), tok.SnowballStemmerBased_Tokenizer()]
}

hyperparams = aux_functions.generate_params_comb_list(**all_hyperparams)          

print('Performing model hyperparameters search...')

def run_model(idx, **hyperp):    
    current_model = LSI(**hyperp)
    current_model.set_name('LSI_Model_{}'.format(idx))
    current_model.recover_links(corpus, query, test_cases_names, bug_reports_names)
    
    evaluator = m_eval.ModelEvaluator(orc.oracle, current_model)
    evaluator.evaluate_model()
    evaluator.dump_model()
    evaluator.dump_evaluator()
    
    return([evaluator.get_mean_precision(), 
            evaluator.get_mean_recall(),
            evaluator.get_mean_fscore(), 
            evaluator.get_model().get_name(),
            evaluator.get_model().get_top_value(),
            evaluator.get_model().get_vectorizer_type(), 
            evaluator.get_model().get_tokenizer_type(),
            evaluator.get_model().get_sim_measure_min_threshold()[0],
            evaluator.get_model().get_sim_measure_min_threshold()[1],
            evaluator.get_model().get_model_dump_path(),
            evaluator.get_evaluator_dump_path()
           ])

tasks = [(idx,hp) for idx,hp in enumerate(hyperparams)]
#results = Parallel(n_jobs=-1, verbose=1)(delayed(run_model)(idx, **hp) for idx,hp in tasks)
results = [run_model(t[0],**t[1]) for t in tasks]
results_df = pd.DataFrame(data=results, 
                          columns=['precision', 'recall', 'fscore', 'model_name', 'top_value', 'vectorizer', 'tokenizer', 'metric', 'metric_value', 'model_dump', 'evaluator_dump'])
results_df = results_df.astype(dtype={'model_dump' : str, 'evaluator_dump' : str})


Performing model hyperparameters search...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
exception calling callback for <Future at 0x7fd117ffee48 state=finished raised BrokenProcessPool>
sklearn.externals.joblib.externals.loky.process_executor._RemoteTraceback: 
'''
Traceback (most recent call last):
  File "/home/guilherme/anaconda3/envs/trace-link-recovery-study/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py", line 391, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/home/guilherme/anaconda3/envs/trace-link-recovery-study/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/home/guilherme/anaconda3/envs/trace-link-recovery-study/lib/python3.6/enum.py", line 135, in __new__
    enum_members = {k: classdict[k] for k in classdict._member_names}
AttributeError: 'dict' object has no attribute '_member_names'
'''

The above exception was the direct cause of the following

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

### Report Best Model

In [51]:
bm = aux_functions.report_best_model(results_df)

------------ Report -------------------

Total of Analyzed Hyperparameters Combinations: 1

Best Model Hyperparameters Combination Found:

{'Measures': {'Mean FScore of LSI_Model_0': 0.0,
              'Mean Precision of LSI_Model_0': 0.0,
              'Mean Recall of LSI_Model_0': 0.0},
 'Setup': [{'Name': 'LSI_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.8)},
           {'Top Value': 10},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 100,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.float64'>,
                           'encoding': 'utf-8',
                           'input': 'content',
                           'lowercase': True

#### Save Similarity Matrix

In [22]:
bm.save_sim_matrix()

#### Best Model for TOP 3 and 5 - Cosine

In [44]:
aux_functions.print_report_top_3_and_5_v3(results_df, SimilarityMeasure.COSINE.value)

{'Measures': {'Mean FScore of LSI_Model_8': 0.5833333333333334,
              'Mean Precision of LSI_Model_8': 0.48809523809523814,
              'Mean Recall of LSI_Model_8': 0.7976190476190476},
 'Setup': [{'Name': 'LSI_Model_8'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 3},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 5,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.int64'>,
                           'encoding': 'utf-8',
                           'input': 'content',
                           'lowercase': True,
                           'max_df': 1.0,
                           'max_features': None,
    